## EPA Flight Database Wescraping

Import webscraping libraries

In [121]:
import requests
from bs4 import BeautifulSoup as BS
import os
import glob
import re

# import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import time
from threading import Thread

# import Action chains  
from selenium.webdriver.common.action_chains import ActionChains 

# import dataframe libraries
import pandas as pd
import numpy as np
from collections import defaultdict # library allows multiple values to be stored under a single key 

Access flight database

In [122]:
# Setup browser
options = Options()
options.add_argument("--headless")  # Keep off for debugging
driver = webdriver.Chrome(service=Service(), options=options)

try:
    url = "https://ghgdata.epa.gov/ghgp/main.do#/facility/?q=Find%20a%20Facility%20or%20Location&st=&bs=&et=&fid=&sf=11001100&lowE=-20000&highE=23000000&g1=1&g2=1&g3=1&g4=1&g5=1&g6=0&g7=1&g8=1&g9=1&g10=1&g11=1&g12=1&s1=1&s2=1&s3=1&s4=1&s5=1&s6=1&s7=1&s8=1&s9=1&s10=1&s201=1&s202=1&s203=1&s204=1&s301=1&s302=1&s303=1&s304=1&s305=1&s306=1&s307=1&s401=1&s402=1&s403=1&s404=1&s405=1&s601=1&s602=1&s701=1&s702=1&s703=1&s704=1&s705=1&s706=1&s707=1&s708=1&s709=1&s710=1&s711=1&s801=1&s802=1&s803=1&s804=1&s805=1&s806=1&s807=1&s808=1&s809=1&s810=1&s901=1&s902=1&s903=1&s904=1&s905=1&s906=1&s907=1&s908=1&s909=1&s910=1&s911=1&si=&ss=&so=0&ds=E&yr=2023&tr=current&cyr=2023&ol=0&sl=0&rs=ALL"  # your full URL
    driver.get(url)

    # Wait a moment for modal to appear (up to 15 seconds)
    try:
        Close_Warning = driver.find_element(By.XPATH, "/html/body/div[7]/div[2]/div[1]/div[3]/img").click()
        print("Modal closed successfully.")
    except Exception as e:
        print("Modal not found or already dismissed:", e)

    # Get page HTML
    html = driver.page_source


finally:
    driver.quit() 

Modal closed successfully.


Find all sector and subsector data based on html criteria

In [123]:
def extract_names(soup_section, style=None, class_list=None):
    """
    Extract only label text (e.g., 'Power Plants') from a section of checkboxes.
    
    :param soup_section: BeautifulSoup element containing the checkboxes
    :param style: optional style string to filter inputs (for sectors)
    :param class_list: optional list of class names to filter (for subsectors)
    :return: list of label names (strings)
    """
    if not soup_section:
        return []

    checkboxes = []

    if style:
        checkboxes = soup_section.find_all('input', style=style)
    elif class_list:
        for class_name in class_list:
            checkboxes.extend(soup_section.find_all('input', class_=class_name))
    else:
        checkboxes = soup_section.find_all('input', type='checkbox')

    names = []
    for checkbox in checkboxes:
        label = checkbox.find_parent('label')
        if label:
            label_text = label.get_text(strip=True)
            names.append(label_text)

    return names

In [124]:
s = BS(html, 'html.parser')
results = s.find(id='filterSectorPopover')

if results:
    # Extract top-level sectors
    All_Sectors = extract_names(results, style="float:left;")

    # Extract Subsectors by known class groups
    subsector_classes = [
        "check check_petro", "check check_chem", "check check_other",
        "check check_mineral", "check check_waste", "check check_metal", "check check_pulp"
    ]
    subsector_names = extract_names(results, class_list=subsector_classes)


Xpaths for sectors and subsectors

In [125]:
value_list = [
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/li[3]/label"]

In [126]:
subsector_list =[
    # Pretroleum and Natural Gas Systems
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/span[1]/li/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[4]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/span[2]/li/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[5]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[6]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[7]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[8]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[1]/ul/div/li[9]/label",


# Chemicals
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[4]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[5]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[6]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[7]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[8]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[9]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[10]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[2]/ul/div/li[11]/label",

# Other 
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[4]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[5]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[6]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[7]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[8]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[9]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[1]/li[10]/label",

# Minerals
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[2]/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[2]/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[2]/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[2]/li[4]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[3]/ul/div[2]/li[5]/label",

# Waste
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[1]/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[1]/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[1]/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[1]/li[4]/label",

# Metals
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[2]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[3]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[4]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[5]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[6]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[2]/li[7]/label",

# Pulp and Paper
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[3]/li[1]/label",
"/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/table/tbody/tr/td[4]/ul/div[3]/li[2]/label"
]

Set up directories for GHG flight datasets

In [127]:
def get_project_root():
    """
    Sets working directory to the notebook's location and returns the project root
    (assumes the notebook is inside the /notebooks directory of your project).
    """
    try:
        # Use IPython magic to get the notebook's current directory
        from IPython import get_ipython
        ipython = get_ipython()

        if ipython:
            notebook_dir = ipython.run_line_magic("pwd", "")
            os.chdir(notebook_dir)
            print(f"Working directory set to: {notebook_dir}")
    except Exception as e:
        print(f"Warning: Failed to change working directory: {e}")

    # Project root is assumed to be one level up from notebooks/
    return os.path.abspath(os.path.join(os.getcwd(), '..'))
def create_output_dirs(base_dir, subfolders):
    """
    Create data output subfolders like Sector and Subsector inside processed/.

    :param base_dir: base path where to create folders (e.g. data/processed)
    :param subfolders: list of folder names
    :return: dict of full paths {folder_name: full_path}
    """
    os.makedirs(base_dir, exist_ok=True)
    paths = {}
    for folder in subfolders:
        path = os.path.join(base_dir, folder)
        os.makedirs(path, exist_ok=True)
        paths[folder] = path
    return paths

In [128]:
# Get base path to project (assumes notebook lives in notebooks/)
project_root = get_project_root()
# # Create output directories
processed_data_root = os.path.join(project_root, 'data', 'raw')
subfolders = ['Sector', 'Subsector']

# Create output directories
paths = create_output_dirs(processed_data_root, subfolders)

# Sector folder 
sector_folder = paths['Sector']
# Subsector folder
subsector_folder = paths['Subsector']


Working directory set to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\notebooks


In [129]:
# Setup path to download folder
home = os.path.expanduser('~')
path = os.path.join(home, 'Downloads')

Sector Data Download


In [130]:
# Chrome Driver
options = Options()
options.add_argument("--headless")  # Keep off for debugging
driver = webdriver.Chrome(service=Service(), options=options)
driver.get(url)

# driver = webdriver.Chrome()
# driver.get(url)

Close_Warning = driver.find_element(By.XPATH, "/html/body/div[7]/div[2]/div[1]/div[3]/img").click()
Filter_Btn = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/button")
Deselect_All = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/input[2]")

Filter_Btn.click()
Deselect_All.click()

for value in value_list:
    
    Values_Sector = driver.find_element(By.XPATH, value).click()

    Export_Data = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[2]/div[3]/span/div/input[2]").click()
    All_Reporting_Year = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[2]/div[3]/span/div/div/a[2]").click()
    ActionChains(driver).move_to_element(Filter_Btn).click(Filter_Btn).perform()
    ActionChains(driver).move_to_element(Deselect_All).click(Deselect_All).perform()
    time.sleep(2) 
    
driver.quit()

List of sector excel files downloaded in to downloads folder

In [131]:
# Get list of all .xls files in Downloads folder
xls_files = glob.glob(os.path.join(path, "*.xls"))

# Sort by modification time (latest last)
xls_files.sort(key=os.path.getmtime, reverse=True)

# Select only the most recent N files, where N = number of sectors selected
recent_files = xls_files[:len(All_Sectors)]  # All_Sectors must come from BeautifulSoup

# Reverse to align with sector iteration order (since you downloaded in sequence)
recent_files.reverse()

# Rename based on sector/subsector name
for i, sector in enumerate(All_Sectors):
    original = recent_files[i]
    renamed = os.path.join(path, f"{sector}.xls")

    try:
        os.rename(original, renamed)
        print(f"Renamed {original} to {renamed}")
    except Exception as e:
        print(f"Error renaming file: {e}")

Renamed C:\Users\jayja\Downloads\flight.xls to C:\Users\jayja\Downloads\Power Plants.xls
Renamed C:\Users\jayja\Downloads\flight (2).xls to C:\Users\jayja\Downloads\Petroleum and Natural Gas Systems.xls
Renamed C:\Users\jayja\Downloads\flight (1).xls to C:\Users\jayja\Downloads\Refineries.xls
Renamed C:\Users\jayja\Downloads\flight (3).xls to C:\Users\jayja\Downloads\Chemicals.xls
Renamed C:\Users\jayja\Downloads\flight (4).xls to C:\Users\jayja\Downloads\Other.xls
Renamed C:\Users\jayja\Downloads\flight (5).xls to C:\Users\jayja\Downloads\Minerals.xls
Renamed C:\Users\jayja\Downloads\flight (6).xls to C:\Users\jayja\Downloads\Waste.xls
Renamed C:\Users\jayja\Downloads\flight (7).xls to C:\Users\jayja\Downloads\Metals.xls
Renamed C:\Users\jayja\Downloads\flight (8).xls to C:\Users\jayja\Downloads\Pulp and Paper.xls


In [132]:
# Get list of all .xls files in Downloads folder
list_of_files = glob.glob(os.path.join(path, "*.xls"))

# Sort by modification time (latest last)
list_of_files.sort(key=os.path.getmtime, reverse=True)


for file in list_of_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    all_df = pd.read_excel(file, header=6,sheet_name=None)

    combined_sheet = []
    for sheet,df in all_df.items():
        # Add a column for the sheet name
        df["INDUSTRY TYPE"] = file_name
        combined_sheet.append(df)  


    combined_df = pd.concat(combined_sheet, ignore_index=True)

    excel_path = os.path.join(sector_folder, f"{file_name}.xlsx")
    combined_df.to_excel(excel_path, index=False)
    print(f"Saved modified file to: {excel_path}")
    # Optionally delete original
    os.remove(file)  # Only if you're sure!

Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Pulp and Paper.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Metals.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Waste.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Minerals.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Other.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Chemicals.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Refineries.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_explorer\ghg_explorer\data\raw\Sector\Petroleum and Natural Gas Systems.xlsx
Saved modified file to: c:\Users\jayja\Documents\projects\ghg_

Subsector Data Download

In [133]:
# Chrome Driver

# Setup browser
options = Options()
options.add_argument("--headless")  # Keep off for debugging
driver = webdriver.Chrome(service=Service(), options=options)
driver.get(url)

# driver = webdriver.Chrome()
# driver.get(url)

Close_Warning = driver.find_element(By.XPATH, "/html/body/div[7]/div[2]/div[1]/div[3]/img").click()
Filter_Btn = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/button")
Deselect_All = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/table/tbody/tr[1]/td/div[3]/table/tbody/tr[1]/td[2]/div/input[2]")

Filter_Btn.click()
Deselect_All.click()

for value in subsector_list:
    
    Values_Sector = driver.find_element(By.XPATH, value).click()

    Export_Data = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[2]/div[3]/span/div/input[2]").click()
    All_Reporting_Year = driver.find_element(By.XPATH, "/html/body/div[7]/div[1]/div[2]/div[3]/span/div/div/a[2]").click()
    ActionChains(driver).move_to_element(Filter_Btn).click(Filter_Btn).perform()
    ActionChains(driver).move_to_element(Deselect_All).click(Deselect_All).perform()
    time.sleep(2) 
    
driver.quit()

List of subsector excel files downloaded in to downloads folder

In [134]:
def sanitize_filename(name):
    # Replace forbidden characters with underscores
    name = re.sub(r'[\\/*?:"<>|]', "_", name)
    # Remove periods
    name = name.replace('.', '')
    return name

In [ ]:
# Get list of all .xls files in Downloads folder
xls_files = glob.glob(os.path.join(path, "*.xls"))

# Sort by modification time (latest last)
xls_files.sort(key=os.path.getmtime, reverse=True)

# Select only the most recent N files, where N = number of sectors selected
recent_files = xls_files[:len(subsector_names)]  # All_Sectors must come from BeautifulSoup

# Reverse to align with sector iteration order (since you downloaded in sequence)
recent_files.reverse()

# Rename based on sector/subsector name
for i, subsector in enumerate(subsector_names):
    safe_name = sanitize_filename(subsector)
    original = recent_files[i]
    renamed = os.path.join(path, f"{safe_name}.xls")

    try:
        os.rename(original, renamed)
        # print(f"Renamed {original} to {renamed}")
    except Exception as e:
        print(f"Error renaming file: {e}")

In [ ]:
# Get list of all .xls files in Downloads folder
list_of_files = glob.glob(os.path.join(path, "*.xls"))

# Sort by modification time (latest last)
list_of_files.sort(key=os.path.getmtime, reverse=True)

Selected_columns = []
for file in list_of_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    all_df = pd.read_excel(file, header=6,sheet_name=None)

    combined_sheet = []
    for sheet,df in all_df.items():
        if not df.dropna(how='all').empty:
            # Add a column for the sheet name
            df["SUBSECTOR_TYPE"] = file_name
            combined_sheet.append(df)

    combined_df = pd.concat(combined_sheet, ignore_index=True)
    
    excel_path = os.path.join(subsector_folder, f"{file_name}.xlsx")
    combined_df.to_excel(excel_path, index=False)
    print(f"Saved modified file to: {excel_path}")

    # Create a new DataFrame with only the selected columns
    Selected_columns.append(combined_df[['GHGRP ID','SUBSECTOR_TYPE']])

Selected_df = pd.concat(Selected_columns, ignore_index=True)
    # Optionally delete original
os.remove(file)

In [142]:
Pivot_df = pd.pivot_table(
    Selected_df,
    index='GHGRP ID',
    aggfunc=lambda x: ",".join(
        sorted(filter(None, map(str.strip, x.dropna().unique())))
    ),
    sort=False
).reset_index()

Pivot_df.to_excel("Pivot_table.xlsx", index=False)

Update Sector Files with subsector information

In [140]:
# Define all sector subsectors
sector_subsectors = {
    "Power Plants": [],
    "Petroleum and Natural Gas Systems": [
        "Offshore Petroleum & Natural Gas Production",
        "Onshore Petroleum & Natural Gas Production",
        "Onshore Petroleum & Natural Gas Gathering & Boosting",
        "Natural Gas Processing",
        "Natural Gas Transmission_Compression",
        "Onshore Natural Gas Transmission Pipelines",
        "Natural Gas Local Distribution Companies",
        "Underground Natural Gas Storage",
        "Liquefied Natural Gas Storage",
        "Liquefied Natural Gas Imp_Exp Equipment",
        "Other Petroleum & Natural Gas Systems"
    ],
    "Refineries": [],
    "Chemicals": [
        "Adipic Acid Production",
        "Ammonia Manufacturing",
        "Fluorinated GHG Production",
        "HCFC-22 Prod_HFC-23 Dest",
        "Hydrogen Production",
        "Nitric Acid Production",
        "Petrochemical Production",
        "Phosphoric Acid Production",
        "Silicon Carbide Production",
        "Titanium Dioxide Production",
        "Other Chemicals"
    ],
    "Other": [
        "Underground Coal Mines",
        "Food Processing",
        "Ethanol Production",
        "Universities",
        "Manufacturing",
        "Military",
        "Use of Electical Equipment",
        "Electronics Manufacturing",
        "Electrical Equipment Manufacturers",
        "Other"
    ],
    "Minerals": [
        "Cement Production",
        "Glass Production",
        "Lime Manufacturing",
        "Soda Ash Manufacturing",
        "Other Minerals"
    ],
    "Waste": [
        "Municipal Landfills",
        "Industrial Landfills",
        "Wastewater Treatment",
        "Solid Waste Combustion"
    ],
    "Metals": [
        "Aluminum Production",
        "Ferroalloy Production",
        "Iron & Steel Production",
        "Lead Production",
        "Magnesium",
        "Zinc Production",
        "Other Metals"
    ],
    "Pulp and Paper": [
        "Pulp and Paper Manufacturers",
        "Other Paper Producers"
    ],
}

In [141]:
# Define the output folder
enriched_folder = os.path.join(sector_folder, "enriched")
os.makedirs(enriched_folder, exist_ok=True)

# Find all .xlsx files
excel_files = glob.glob(os.path.join(sector_folder, "*.xlsx"))

for file in excel_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    current_sector = file_name.strip()

    if current_sector not in sector_subsectors:
        print(f"Skipping {file_name} (not found in sector_subsectors)")
        continue

    # Prepare comparison dictionaries for the sector
    valid_subsectors_cf = {s.casefold().strip() for s in sector_subsectors[current_sector]}
    original_subsector_map = {s.casefold().strip(): s for s in sector_subsectors[current_sector]}

    # Load Excel
    df = pd.read_excel(file)
    df.columns = df.columns.str.strip()

    # Drop old SUBSECTOR_TYPE if it exists to avoid merge column conflicts
    if 'SUBSECTOR_TYPE' in df.columns:
        df.drop(columns=['SUBSECTOR_TYPE'], inplace=True)

    # Merge with Pivot_df
    merged_df = df.merge(Pivot_df, on='GHGRP ID', how='left')

    # Filtering function
    def filter_subsectors(val):
        if pd.isna(val):
            return None
        subsectors = [s.strip().casefold() for s in val.split(',')]
        filtered = [s for s in subsectors if s in valid_subsectors_cf]
        readable = [original_subsector_map[s] for s in filtered]
        return ",".join(readable) if readable else None

    # Apply filtering
    if 'SUBSECTOR_TYPE' in merged_df.columns:
        merged_df['SUBSECTOR_TYPE'] = merged_df['SUBSECTOR_TYPE'].apply(filter_subsectors)
    elif 'SUBSECTOR_TYPE_y' in merged_df.columns:
        merged_df['SUBSECTOR_TYPE'] = merged_df['SUBSECTOR_TYPE_y'].apply(filter_subsectors)
        merged_df.drop(columns=['SUBSECTOR_TYPE_x', 'SUBSECTOR_TYPE_y'], errors='ignore', inplace=True)
    else:
        print(f"Skipping {file_name} — no usable SUBSECTOR_TYPE column found")
        continue

    # Save to enriched output folder
    enriched_path = os.path.join(enriched_folder, f"{file_name}.xlsx")
    merged_df.to_excel(enriched_path, index=False)                               